In [ ]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -qU Pillow langchain_community tiktoken langchain-openai chromadb langchain audiocraft torchaudio ffmpeg-python pillow

In [ ]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload an image

Saving TARS_Playing_music.webp to TARS_Playing_music.webp


In [ ]:
from PIL import Image as PILImage
from IPython.display import display

# Open the webp image
img = PILImage.open('TARS_Playing_music.webp')

# Convert and save as a png file
img.save('TARS_Playing_music.png', 'PNG')

# Now display the PNG image
display(PILImage.open('TARS_Playing_music.png'))


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import os
import time
import random
import json
import torchaudio
import ipywidgets as widgets
from PIL import Image
from IPython.display import display, Audio
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
import openai


# LLM models and Keys to access them

In [ ]:
# Select model option
model_option = input("Choose a model (gpt-4o-mini / gpt-3.5-turbo): ")
if model_option == 'gpt-4o-mini':
    model_name = 'gpt-4o-mini'
elif model_option == 'gpt-3.5-turbo':
    model_name = 'gpt-3.5-turbo'
else:
    model_name = 'gpt-3.5-turbo'  # Default to gpt-3.5-turbo

# Load keys from userdata (if available)
openai_key = userdata.get('OPENAI_API_KEY')
langchain_key = userdata.get('LANGCHAIN_API_KEY')
huggingface_key = userdata.get('HuggingFace_read')

# If keys are not found in userdata, prompt the user to input them
if not openai_key:
    openai_key = input("Enter your OpenAI API key: ")
if not langchain_key:
    langchain_key = input("Enter your Langchain API key: ")
if not huggingface_key:
    huggingface_key = input("Enter your Hugging Face API key: ")

# Set the API keys globally
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["LANGCHAIN_API_KEY"] = langchain_key
os.environ["HUGGINGFACE_API_KEY"] = huggingface_key

# Confirm which keys were loaded
print("OpenAI, Langchain, and Hugging Face API keys have been loaded.")


Choose a model (gpt-4o-mini / gpt-3.5-turbo): gpt-4o-mini
OpenAI, Langchain, and Hugging Face API keys have been loaded.


In [ ]:
model_name

'gpt-4o-mini'

In [ ]:
# Set the folder path where images are located
input_folder = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/20241005ec0a6de5f499c06116be38018e3d4fb455cd59c50931f15f812b7e76de97e7c3'
output_folder = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/resized_1_4th'
scale_factor = 0.25  # 0.25 for 1/4th size, 0.125 for 1/8th size

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path)

        # Resize the image
        new_size = (int(img.width * scale_factor), int(img.height * scale_factor))
        resized_img = img.resize(new_size)

        # Save resized image to the output folder
        resized_img.save(os.path.join(output_folder, filename))

print("All images resized and saved.")


All images resized and saved.


# Text description of the image using GPT-4o

In [ ]:
# Function to get an emotional, astronomy-focused description using GPT-4 via LangChain
def describe_image_with_emotion(image_path):
    try:
        # Refined prompt for astronomy enthusiasts
        prompt_template = """
        You are an AI astronomer specializing in deep space photography from the James Webb Space Telescope.
        Describe the celestial objects, cosmic structures, and phenomena in the image. Focus on the scientific awe, the grandeur of space,
        and the emotions these distant galaxies, nebulae, or stars might evoke for astronomy enthusiasts.

        Image: {image_path}
        """
        # Use ChatOpenAI instead of OpenAI, temperature adjusted for creativity
        llm = ChatOpenAI(temperature=0.7, model=model_name)
        template = PromptTemplate(input_variables=["image_path"], template=prompt_template)
        chain = LLMChain(llm=llm, prompt=template)

        # Run the chain and return the result
        return chain.run(image_path)
    except Exception as e:
        return f"Error in generating emotional description: {e}"

# Process all images in a directory
def process_images_in_directory(directory_path):
    image_descriptions = {}

    # Iterate through each file in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Check if it's an image file
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp')):
            # Get emotional description using GPT-4 via LangChain
            emotional_description = describe_image_with_emotion(file_path)
            if "Error" in emotional_description:
                print(f"Skipping {filename} due to GPT-4 error: {emotional_description}")
                continue

            # Store the result
            image_descriptions[filename] = emotional_description

            # Sleep for 5 seconds before making the next request
            time.sleep(5)

    return image_descriptions

# Example: Process all images in the directory
directory_path = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/Selected5Images'
image_descriptions_emotional = process_images_in_directory(directory_path)

# Save the descriptions to a file
output_path = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/image_descriptions_gpt-4o.txt'
with open(output_path, 'w') as f:
    for image_name, description in image_descriptions_emotional.items():
        f.write(f"{image_name}: {description}\n")

print(f"Descriptions saved to {output_path}")


Descriptions saved to /content/drive/MyDrive/Hackathons/NASASpaceChallenge/image_descriptions_gpt-4o.txt


In [ ]:
import os
import time
import json
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# Function to get an emotional, astronomy-focused description using GPT-4 via LangChain
def describe_image_with_emotion(image_path):
    try:
        # Refined prompt for astronomy enthusiasts with description length control
        prompt_template = """
        You are an AI astronomer specializing in deep space photography from the James Webb Space Telescope.
        Provide a concise description of the celestial objects, cosmic structures, and phenomena in the image.
        Keep the description detailed but between 200 and 300 characters. Focus on awe-inspiring elements for astronomy enthusiasts.

        Image: {image_path}
        """
        # Use ChatOpenAI with model 'gpt-4o-mini', temperature adjusted for creativity
        llm = ChatOpenAI(temperature=0.7, model="gpt-4o-mini")
        template = PromptTemplate(input_variables=["image_path"], template=prompt_template)
        chain = LLMChain(llm=llm, prompt=template)

        # Run the chain and return the result
        description = chain.run(image_path)
        return description
    except Exception as e:
        return f"Error in generating emotional description: {e}"

# Process all images in a directory
def process_images_in_directory(directory_path):
    # Set the output directory for descriptions
    output_directory = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions'
    os.makedirs(output_directory, exist_ok=True)

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Check if it's an image file
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp')):
            # Get emotional description using GPT-4 via LangChain
            emotional_description = describe_image_with_emotion(file_path)
            if "Error" in emotional_description:
                print(f"Skipping {filename} due to GPT-4 error: {emotional_description}")
                continue

            # Make sure image name is properly defined
            image_name, _ = os.path.splitext(filename)

            # Define the output path for JSON
            output_path_json = os.path.join(output_directory, f"{image_name}.json")

            # Save to JSON file
            with open(output_path_json, 'w') as json_file:
                json.dump({image_name: emotional_description}, json_file)

            # Sleep for 1 second before making the next request
            time.sleep(1)

# Example: Process all images in the directory
directory_path = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/Selected5Images'
process_images_in_directory(directory_path)

print("Descriptions saved.")


Descriptions saved.


# **Music Generation**

In [ ]:
# Initialize MusicGen model
model_music = MusicGen.get_pretrained('facebook/musicgen-large')
model_music.set_generation_params(duration=30)  # Generate 'x' seconds of audio

# Predefined options for genre, style, and instrument
genres = ["Experimental", "Classical", "Ambient", "Jazz", "Rock"]
styles = ["Noise", "Melodic", "Rhythmic", "Atmospheric", "Harmonic"]
instruments = ["Modular Synth", "Piano", "Guitar", "Drums", "Violin"]

# Dropdown widgets for genre, style, and instrument selection
genre_dropdown = widgets.Dropdown(options=genres, description='Genre:')
style_dropdown = widgets.Dropdown(options=styles, description='Style:')
instrument_dropdown = widgets.Dropdown(options=instruments, description='Instrument:')

# Dropdown widget for description file selection
input_directory = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions'
json_files = [f for f in os.listdir(input_directory) if f.endswith('.json')]
description_dropdown = widgets.Dropdown(options=json_files, description='Description File:')
confirm_button = widgets.Button(description="Generate Music")

# Function to handle the confirm button click
def on_button_click(b):
    selected_genre = genre_dropdown.value
    selected_style = style_dropdown.value
    selected_instrument = instrument_dropdown.value
    selected_file = description_dropdown.value
    selected_file_path = os.path.join(input_directory, selected_file)

    # Load the description
    with open(selected_file_path, 'r') as f:
        description_data = json.load(f)

    # Access the correct key dynamically (image name)
    image_name = list(description_data.keys())[0]
    original_description = description_data[image_name]

    # Append the selected genre, style, and instrument as a prompt to the original description
    music_prompt = f"{original_description} Set to the {selected_genre} genre, with a {selected_style} style, and featuring the {selected_instrument} as the primary instrument."

    print(f"Generating music... for {image_name}")

    # Generate music using MusicGen
    wav = model_music.generate([music_prompt])

    # Save each generated audio with torchaudio, using selected options as a prefix
    for idx, one_wav in enumerate(wav):
        # Create a filename with selected genre, style, instrument, and index
        # Create a filename with image_name, selected genre, style, instrument, and index
        filename_prefix = f"{image_name}_{selected_genre}_{selected_style}_{selected_instrument}_{idx}"
        filename = os.path.join(input_directory, f'{filename_prefix}_music.wav')


        # Save the audio file in the same directory
        torchaudio.save(filename, one_wav.cpu(), sample_rate=model_music.sample_rate)
        display(Audio(filename, autoplay=False))  # Display the audio for playback

    print(f"Music generated and saved.")

# Confirm button click event
confirm_button.on_click(on_button_click)

# Display dropdowns and confirm button
display(genre_dropdown, style_dropdown, instrument_dropdown, description_dropdown, confirm_button)



Output hidden; open in https://colab.research.google.com to view.

# **Music generation with more elements**

In [ ]:
import os
import json
import torchaudio
from audiocraft.models import MusicGen
import ipywidgets as widgets
from IPython.display import display, Audio

# Initialize MusicGen model
model_music = MusicGen.get_pretrained('facebook/musicgen-large')
model_music.set_generation_params(duration=30)  # Generate 30 seconds of audio

# Expanded predefined options for genre, style, instrument, tempo, mood, and cultural style
genres = ["Experimental", "Classical", "Ambient", "Jazz", "Rock", "Arabic", "Indian", "Chinese", "Latin"]
styles = ["Noise", "Melodic", "Rhythmic", "Atmospheric", "Harmonic", "Traditional", "Modern", "Fusion"]
instruments = ["Modular Synth", "Piano", "Guitar", "Drums", "Violin", "Sitar", "Tabla", "Erhu", "Oud", "Darbuka", "Flute"]
tempos = ["Slow", "Medium", "Fast"]
moods = ["Happy", "Sad", "Mysterious", "Energetic", "Relaxing", "Uplifting"]

# Dropdown widgets for genre, style, instrument, tempo, and mood selection
genre_dropdown = widgets.Dropdown(options=genres, description='Genre:')
style_dropdown = widgets.Dropdown(options=styles, description='Style:')
instrument_dropdown = widgets.Dropdown(options=instruments, description='Instrument:')
tempo_dropdown = widgets.Dropdown(options=tempos, description='Tempo:')
mood_dropdown = widgets.Dropdown(options=moods, description='Mood:')

# Dropdown widget for description file selection
input_directory = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions'
json_files = [f for f in os.listdir(input_directory) if f.endswith('.json')]
description_dropdown = widgets.Dropdown(options=json_files, description='Description File:')
confirm_button = widgets.Button(description="Generate Music")

# Function to handle the confirm button click
def on_button_click(b):
    # Fetch selected options
    selected_genre = genre_dropdown.value
    selected_style = style_dropdown.value
    selected_instrument = instrument_dropdown.value
    selected_tempo = tempo_dropdown.value
    selected_mood = mood_dropdown.value
    selected_file = description_dropdown.value
    selected_file_path = os.path.join(input_directory, selected_file)

    # Load the description
    with open(selected_file_path, 'r') as f:
        description_data = json.load(f)

    # Access the correct key dynamically (image name)
    image_name = list(description_data.keys())[0]
    original_description = description_data[image_name]

    # Create the prompt for music generation
    music_prompt = f"{original_description} Set to the {selected_genre} genre, with a {selected_style} style, featuring {selected_instrument}, at a {selected_tempo} tempo, and a {selected_mood} mood."

    print(f"Generating music... for {image_name}")

    # Generate music using MusicGen
    wav = model_music.generate([music_prompt])

    # Create a filename with the selections and image name
    filename_prefix = f"{image_name}_{selected_genre}_{selected_style}_{selected_instrument}_{selected_tempo}_{selected_mood}"
    filename = os.path.join(input_directory, f'{filename_prefix}_music.wav')

    # Save the generated audio file
    torchaudio.save(filename, wav[0].cpu(), sample_rate=model_music.sample_rate)

    # Display the audio for playback
    display(Audio(filename, autoplay=False))

    print(f"Music generated and saved as {filename_prefix}_music.wav")

# Confirm button click event
confirm_button.on_click(on_button_click)

# Display dropdowns and confirm button
display(genre_dropdown, style_dropdown, instrument_dropdown, tempo_dropdown, mood_dropdown,
        description_dropdown, confirm_button)


Output hidden; open in https://colab.research.google.com to view.

# **Music prompt enhancement with another call to GPT4o**

In [ ]:
# Initialize MusicGen model
model_music = MusicGen.get_pretrained('facebook/musicgen-large')
model_music.set_generation_params(duration=60)  # Generate 30 seconds of audio
music_prompt = None
image_name=None

# Expanded predefined options for genre, style, instrument, tempo, mood, and cultural style
genres = ["Experimental", "Classical", "Ambient", "Jazz", "Rock", "Arabic", "Indian", "Chinese", "Latin"]
styles = ["Noise", "Melodic", "Rhythmic", "Atmospheric", "Harmonic", "Traditional", "Modern", "Fusion"]
instruments = ["Modular Synth", "Piano", "Guitar", "Drums", "Violin", "Sitar", "Tabla", "Erhu", "Oud", "Darbuka", "Flute"]
tempos = ["Slow", "Medium", "Fast"]
moods = ["Happy", "Sad", "Mysterious", "Energetic", "Relaxing", "Uplifting"]

# Dropdown widgets for genre, style, instrument, tempo, and mood selection
genre_dropdown = widgets.Dropdown(options=genres, description='Genre:')
style_dropdown = widgets.Dropdown(options=styles, description='Style:')
instrument_dropdown = widgets.Dropdown(options=instruments, description='Instrument:')
tempo_dropdown = widgets.Dropdown(options=tempos, description='Tempo:')
mood_dropdown = widgets.Dropdown(options=moods, description='Mood:')

# Dropdown widget for description file selection
input_directory = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions'
json_files = [f for f in os.listdir(input_directory) if f.endswith('.json')]
description_dropdown = widgets.Dropdown(options=json_files, description='Description File:')
confirm_button = widgets.Button(description="Generate Music")

# Function to handle the confirm button click
def on_button_click(b):
    global music_prompt, image_name
    # Fetch selected options
    selected_genre = genre_dropdown.value
    selected_style = style_dropdown.value
    selected_instrument = instrument_dropdown.value
    selected_tempo = tempo_dropdown.value
    selected_mood = mood_dropdown.value
    selected_file = description_dropdown.value
    selected_file_path = os.path.join(input_directory, selected_file)

    # Load the description
    with open(selected_file_path, 'r') as f:
        description_data = json.load(f)

    # Access the correct key dynamically (image name)
    image_name = list(description_data.keys())[0]
    original_description = description_data[image_name]

    llm = ChatOpenAI(model_name=model_name, temperature=0.7)

    # Function to enhance the music prompt using ChatOpenAI from Langchain
    def enhance_music_prompt(description, genre, style, instrument, tempo, mood):
        global music_prompt
        # Create the prompt dynamically
        prompt = (f"Given the following image description:\n\n'{description}'\n\n"
              f"Create a creative, descriptive prompt for generating music in the {genre} genre, "
              f"with a {style} style, featuring the {instrument}, at a {tempo} tempo, "
              f"and reflecting a {mood} mood. The generated music prompt should be between 200 and 300 characters in length. "
              f"Ensure the music prompt flows naturally and artistically, while staying within the character limit.")

        # Call the model via Langchain to generate an enhanced prompt
        response = llm.predict(prompt)

        return response.strip()

    # Use the function before generating the music
    music_prompt = enhance_music_prompt(
        original_description, selected_genre, selected_style,
        selected_instrument, selected_tempo, selected_mood
    )

    print(f"Generating music... for {image_name}")

    # Display or print the final generated prompt
    print(f"Final Music prompt: {music_prompt}")

    # Generate music using MusicGen
    wav = model_music.generate([music_prompt])

    # Create a filename with the selections and image name
    filename_prefix = f"{image_name}_{selected_genre}_{selected_style}_{selected_instrument}_{selected_tempo}_{selected_mood}"
    filename = os.path.join(input_directory, f'{filename_prefix}_music.wav')

    # Save the generated audio file
    torchaudio.save(filename, wav[0].cpu(), sample_rate=model_music.sample_rate)

    # Display the audio for playback
    display(Audio(filename, autoplay=False))

    print(f"Music generated and saved as {filename_prefix}_music.wav")

# Confirm button click event
confirm_button.on_click(on_button_click)

# Display dropdowns and confirm button
display(genre_dropdown, style_dropdown, instrument_dropdown, tempo_dropdown, mood_dropdown,
        description_dropdown, confirm_button)


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Dropdown(description='Genre:', options=('Experimental', 'Classical', 'Ambient', 'Jazz', 'Rock', 'Arabic', 'Ind…

Dropdown(description='Style:', options=('Noise', 'Melodic', 'Rhythmic', 'Atmospheric', 'Harmonic', 'Traditiona…

Dropdown(description='Instrument:', options=('Modular Synth', 'Piano', 'Guitar', 'Drums', 'Violin', 'Sitar', '…

Dropdown(description='Tempo:', options=('Slow', 'Medium', 'Fast'), value='Slow')

Dropdown(description='Mood:', options=('Happy', 'Sad', 'Mysterious', 'Energetic', 'Relaxing', 'Uplifting'), va…

Dropdown(description='Description File:', options=('GalacticDance.json', 'PurpleCosmicGlow.json', 'FieryGalact…

Button(description='Generate Music', style=ButtonStyle())

Generating music... for FieryGalacticCore
Final Music prompt: Compose an energetic piece in the Experimental genre, featuring the Violin as the lead instrument. Infuse fast-paced, swirling melodies that mimic the vibrant activity of a galactic core, with fiery accents and intricate patterns echoing stellar birth. Let the music capture the relentless creativity of the cosmos.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 9.06 MiB is free. Process 17382 has 14.74 GiB memory in use. Of the allocated memory 14.25 GiB is allocated by PyTorch, and 358.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
image_name

'PurpleCosmicGlow'

# **Lyrics generation with another call to GPT4o using music prompt and song style**

In [ ]:
llm = ChatOpenAI(model_name=model_name, temperature=0.8)

# Widgets for lyrics customization (reduced to 3)
song_type_dropdown = widgets.Dropdown(options=["Rap", "Poetic", "Narrative"], description='Song Type:')
vocal_style_dropdown = widgets.Dropdown(options=["Melodic", "Chanting", "Fast-paced"], description='Vocal Style:')
lyrics_mood_dropdown = widgets.Dropdown(options=["Uplifting", "Mysterious", "Reflective"], description='Lyrics Mood:')

# Function to generate lyrics prompt using the music_prompt
def generate_lyrics_prompt(music_prompt, song_type, vocal_style, lyrics_mood):
    prompt = (f"Using the following music description:\n\n'{music_prompt}'\n\n"
                  f"Generate creative and concise lyrics for a {song_type} song. The lyrics should match a {vocal_style} vocal style "
                  f"and reflect a {lyrics_mood} mood. Keep the lyrics suitable for a one-minute song, no more than 8-12 lines, "
                  f"and focus on the themes of astronomical elements in {music_prompt}. Avoid repetition and be imaginative.")

    # Call the model via Langchain to generate the lyrics
    response = llm.predict(prompt)

    return response.strip()

# Function to save lyrics to a JSON file
def save_lyrics_to_json(lyrics, image_name, song_type, vocal_style, lyrics_mood):
    filename = f"{image_name}_{song_type}_{vocal_style}_{lyrics_mood}_lyrics.json"
    filepath = os.path.join("/content/drive/MyDrive/Hackathons/NASASpaceChallenge/GeneratedMusic/Lyrics", filename)  # Update path accordingly

    # Save lyrics in JSON format
    with open(filepath, 'w') as f:
        json.dump({"lyrics": lyrics}, f)

    print(f"Lyrics saved to {filepath}")

# Use the music_prompt and selections to generate and save the lyrics
def on_generate_lyrics_click(b):
    # Assume music_prompt is already generated
    final_lyrics = generate_lyrics_prompt(
        music_prompt,
        song_type_dropdown.value,
        vocal_style_dropdown.value,
        lyrics_mood_dropdown.value
    )

    print(f"Generated Lyrics:\n{final_lyrics}")

    # Save the lyrics to a JSON file
    save_lyrics_to_json(
        final_lyrics,
        image_name,  # Assume image_name is already defined
        song_type_dropdown.value,
        vocal_style_dropdown.value,
        lyrics_mood_dropdown.value
    )

# Button to generate the lyrics
generate_lyrics_button = widgets.Button(description="Generate Lyrics")
generate_lyrics_button.on_click(on_generate_lyrics_click)

# Display the widgets and button
display(song_type_dropdown, vocal_style_dropdown, lyrics_mood_dropdown, generate_lyrics_button)


Dropdown(description='Song Type:', options=('Rap', 'Poetic', 'Narrative'), value='Rap')

Dropdown(description='Vocal Style:', options=('Melodic', 'Chanting', 'Fast-paced'), value='Melodic')

Dropdown(description='Lyrics Mood:', options=('Uplifting', 'Mysterious', 'Reflective'), value='Uplifting')

Button(description='Generate Lyrics', style=ButtonStyle())

Generated Lyrics:
**Verse 1**  
In the cradle of the night, where stardust dreams ignite,  
Colors swirl in cosmic dance, painting skies with every glance.  
From the whispers of the void, a symphony is born,  
Guitar flames like shooting stars, a brand new dawn adorns.  

**Chorus**  
Feel the pulse of the universe, rising high and free,  
Melodies of creation, echoing endlessly.  
Nebulae in bloom, a kaleidoscope of light,  
Together we’re igniting the magic of the night.  

**Verse 2**  
Galaxies in motion, with rhythms that entwine,  
Every note a spark of life, a love song so divine.  
As we soar through endless skies, let the music guide our way,  
In this cosmic tapestry, we’ll shine brighter every day.
Lyrics saved to /content/drive/MyDrive/Hackathons/NASASpaceChallenge/GeneratedMusic/Lyrics/PurpleCosmicGlow_Narrative_Melodic_Uplifting_lyrics.json


# Song generated with Suno.ai can be found here
https://sunoai.ai/music/9874d13bd7db491b86e27736cfd8c37b

In [ ]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload an image

Saving SunoAI.webp to SunoAI.webp


In [ ]:
# Open the webp image
img = PILImage.open('SunoAI.webp')

# Convert and save as a png file
img.save('SunoAI.png', 'PNG')

# Now display the PNG image
display(PILImage.open('SunoAI.png'))


Output hidden; open in https://colab.research.google.com to view.